In [33]:

#这个是对的
import os
import nltk
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import random
from nltk.corpus import stopwords
import operator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer


stopwords = nltk.corpus.stopwords.words('english')
new_stopwords = ['/','(',')','{','}','@','|',';','\n','#','+','_','.',',','``',"''",':','-']
stopwords.extend(new_stopwords)

lemmatizer = nltk.stem.WordNetLemmatizer()


path = os.getcwd() + "\\bbc"
list_catagory_files = os.listdir(path)
print(list_catagory_files)
num_catagory = len(list_catagory_files)



def text_label_list(cata_path):
    label = 0
    all_text_list = []
    all_label_list = []
    catagory_file_num = []
    
    for catagory in list_catagory_files:
        catagory_file = path + "\\" + catagory
        print(catagory_file)
        if catagory_file.endswith('.TXT'):
            continue
        else:
        
            file_num = len(os.listdir(catagory_file))
            catagory_file_num.append(file_num)
            print(file_num)
            print(catagory)
            for i in range(1,file_num + 1):
                all_label_list.append(label)
                j = "%03d" % i
                with open("\\".join([catagory_file, str(j) + ".txt"])) as fp:
                    lines = fp.read()
                    article = []
                    text_ = ""
                    text_ += lines
                    article.append(text_)    
                        
                        
                    all_text_list.append(article)
        label += 1
    return all_text_list, all_label_list, catagory_file_num

all_text_list_, all_label_list_, catagory_file_num = text_label_list(list_catagory_files)



        
# get full dataset in order to split our dataset into training and test
dataset_full=[] 
for text, label in zip(all_text_list_, all_label_list_):
    dataset_full.append((text,label))
        

def get_train_test_split(dataset_full,ratio):
    pre_train_set=[]
    pre_test_set=[]
    
    size_dataset_full = len(dataset_full)
    test_size = int(round(size_dataset_full * ratio))
    list_test_indices=random.sample(range(size_dataset_full), test_size)
    
    for i,text in enumerate(dataset_full):
        if i in list_test_indices:
            pre_test_set.append(text)
        else:pre_train_set.append(text)
        
    return pre_train_set,pre_test_set
# get training set, test set
train_set, test_set = get_train_test_split(dataset_full,0.2)
print ("Size training set: "+str(len(train_set)))
print ("Size test set: "+str(len(test_set)))


# get training set, test set, development set
new_test_set, new_dev_set = get_train_test_split(test_set,0.5)

new_train_set=train_set
random.shuffle(new_train_set)
random.shuffle(new_dev_set)
random.shuffle(new_test_set)
print ("Size training set: "+str(len(new_train_set)))
print ("Size test set: "+str(len(new_test_set)))
print ("Size development set: "+str(len(new_dev_set)))





def get_hash_vec(training_set):
    
    hashing_vectorizer = HashingVectorizer(n_features = 20)
    text_all_vec = hashing_vectorizer.transform(training_set).toarray()
    
    return text_all_vec


        
def chi_square_select(X_train,Y_train,num_features):
    fs_sentanalysis = SelectKBest(chi2, k=num_features).fit(X_train, Y_train)
    X_train_new = fs_sentanalysis.transform(X_train)
    return fs_sentanalysis, X_train_new


def mutual_info_select(X_train,Y_train,num_features):
    mfs_sentanalysis = SelectKBest(mutual_info_classif, k=num_features).fit(X_train, Y_train)
    X_train_new = mfs_sentanalysis.transform(X_train)
    return mfs_sentanalysis, X_train_new
    


# def train_svm_classifier(training_set, bigram_vectorizer, chi_num, mutual_num): # Function for training our svm classifier
#     X_train=[]
#     Y_train=[]
#     for instance in training_set:
#         vector_instance_bigram=get_bigram_vec(instance[0],bigram_vectorizer)
#         X_train.append(vector_instance_bigram)
#         Y_train.append(instance[1])
        
#     fs_sentanalysis, X_train_chi_vector = chi_square_select(X_train,Y_train,chi_num)
#     mfs_sentanalysis, X_train_mut_vector = mutual_info_select(X_train_chi_vector,Y_train,mutual_num)
    
#   # Finally, we train the SVM classifier 
#     svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
#     svm_clf.fit(X_train_mut_vector,Y_train)
#     return svm_clf, fs_sentanalysis, mfs_sentanalysis


# # print(all_text_list[-1:])
# print(all_text_list_[1313])
# print(all_label_list_[1313])


# def train_svm_classifier_test(tra_set): # Function for training our svm classifier
X_train_=[]
Y_train_=[]
for instance in new_train_set:
        
    vector_instance_hash =get_hash_vec(instance[0])
#         vector_instance_hash = vector_instance_hash.reshape(-1, 1)
    X_train_.append(vector_instance_hash)
    Y_train_.append(instance[1])
        

X_train_=np.asarray(X_train)
Y_train_=np.asarray(Y_train)
nsamples, nx, ny = X_train.shape
d2_X_train = X_train.reshape((nsamples,nx*ny))
# X_train_= X_train_.reshape(-1,1)    
print(d2_X_train.shape)
# print(Y_train.shape)
    

    
    
  # Finally, we train the SVM classifier 
svm_clf=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf.fit(d2_X_train,Y_train)







# svm_clf, fs_sentanalysis, mfs_sentanalysis = train_svm_classifier(new_train_set, bigram_vec,500, 250)




X_test=[]
Y_test=[]
for instance in new_test_set:   #注意是test_set,上面的描述也错了
    vector_instance=get_hash_vec(instance[0])
    X_test.append(vector_instance)
    Y_test.append(instance[1])
X_test=np.asarray(X_test)
nsamples_, nx_, ny_ = X_test.shape
d2_X_test = X_test.reshape((nsamples_,nx*ny_))
Y_test_gold=np.asarray(Y_test)


from sklearn.metrics import classification_report

Y_text_predictions = svm_clf.predict(d2_X_test)
print(Y_text_predictions)
print(classification_report(Y_test_gold, Y_text_predictions))


['business', 'entertainment', 'politics', 'README.TXT', 'sport', 'tech']
C:\Users\c2008016\Desktop\Programs\coursework\bbc\business
510
business
C:\Users\c2008016\Desktop\Programs\coursework\bbc\entertainment
386
entertainment
C:\Users\c2008016\Desktop\Programs\coursework\bbc\politics
417
politics
C:\Users\c2008016\Desktop\Programs\coursework\bbc\README.TXT
C:\Users\c2008016\Desktop\Programs\coursework\bbc\sport
511
sport
C:\Users\c2008016\Desktop\Programs\coursework\bbc\tech
401
tech
Size training set: 1780
Size test set: 445
Size training set: 1780
Size test set: 223
Size development set: 222
(1780, 20)
[3 1 0 2 4 3 3 3 3 1 3 3 0 0 2 4 4 4 1 3 3 3 3 4 3 3 3 0 4 3 4 2 0 0 1 0 0
 4 2 0 4 3 1 3 0 3 1 4 0 3 0 1 1 4 3 3 0 0 0 4 0 1 4 3 0 4 3 4 1 2 2 2 0 1
 2 2 3 3 2 4 0 2 0 3 3 3 0 2 0 3 0 1 0 2 4 3 3 4 0 4 2 2 0 0 3 1 0 2 1 0 3
 1 2 0 2 2 0 0 3 4 2 4 4 0 1 3 3 4 4 1 1 1 4 3 3 2 4 2 4 2 3 0 4 4 1 0 0 3
 0 4 0 4 1 3 0 2 4 4 2 4 3 0 4 4 2 3 0 3 4 0 1 4 3 1 4 2 0 0 4 0 3 4 0 2 1
 2 0 4 0 3 1

In [28]:

vecc = get_hash_vec(all_text_list_[0])
print(vecc)
print(vecc.shape)
X_train = []
Y_train = []
for instance in new_train_set:
    vector_instance_hash =get_hash_vec(instance[0])
#         vector_instance_hash = vector_instance_hash.reshape(-1, 1)
    X_train.append(vector_instance_hash)
    Y_train.append(instance[1])

X_train=np.asarray(X_train)
print(X_train[0])
print(X_train.shape)

[[-0.1221125   0.39686563 -0.15264063  0.1221125  -0.18316875 -0.244225
   0.21369688  0.15264063 -0.1221125   0.33580938 -0.18316875  0.39686563
   0.18316875  0.09158438 -0.15264063  0.03052813  0.27475313 -0.03052813
  -0.39686563 -0.09158438]]
(1, 20)
[[ 0.04658412  0.12810632  0.11646029 -0.17469044 -0.09316824 -0.11646029
  -0.04658412 -0.03493809  0.1979825   0.24456662 -0.06987618  0.31444279
  -0.01164603 -0.03493809  0.09316824 -0.02329206  0.43090309  0.13975235
  -0.69876177  0.06987618]]
(1780, 1, 20)
